# Custom Loss

In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd

### Huber Loss

In [7]:
x=np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0])
y=np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0])

In [2]:
model=tf.keras.models.Sequential([tf.keras.layers.Dense(1,input_shape=[1])])
model.compile(optimizer=tf.keras.optimizers.SGD(),loss='mse')
model.fit(x,y,epochs=100)

In [11]:
model.predict([10])

array([[17.823782]], dtype=float32)

In [17]:
def huber_loss(y,y_pred):
    threshold=1
    error=y-y_pred
    is_small_error = tf.abs(error) <= threshold
    small_error_loss=tf.square(error)/2
    big_error_loss=threshold*(tf.abs(error)-0.8*threshold)
    return tf.where(is_small_error, small_error_loss, big_error_loss)

In [1]:
model=tf.keras.models.Sequential([tf.keras.layers.Dense(1,input_shape=[1])])
model.compile(optimizer=tf.keras.optimizers.SGD(),loss=huber_loss)
model.fit(x,y,epochs=100)

In [19]:
model.predict([10])

array([[11.411013]], dtype=float32)

### Hyperparameter

#### function

In [31]:
def huber_loss_thr(threshold=1):
    def huber_loss(y,y_pred):
        error=y-y_pred
        is_small_error = tf.abs(error) <= threshold
        small_error_loss=tf.square(error)/2
        big_error_loss=threshold*(tf.abs(error)-0.8*threshold)
        return tf.where(is_small_error, small_error_loss, big_error_loss)
    return huber_loss

In [3]:
model=tf.keras.models.Sequential([tf.keras.layers.Dense(1,input_shape=[1])])
model.compile(optimizer=tf.keras.optimizers.SGD(),loss=huber_loss_thr(threshold=0.2))
model.fit(x,y,epochs=100)

In [35]:
model.predict([10])

array([[16.039885]], dtype=float32)

#### class

In [40]:
from tensorflow.keras.losses import Loss
class MyHuberLoss(Loss):
    threshold=1
    def __init__(self,threshold):
        super().__init__()
        self.threshold=threshold

    def call(self,y,y_pred):
        error=y-y_pred
        is_small_error = tf.abs(error) <= self.threshold
        small_error_loss=tf.square(error)/2
        big_error_loss=self.threshold*(tf.abs(error)-0.8*self.threshold)
        return tf.where(is_small_error, small_error_loss, big_error_loss)

In [4]:
model=tf.keras.models.Sequential([tf.keras.layers.Dense(1,input_shape=[1])])
model.compile(optimizer=tf.keras.optimizers.SGD(),loss=MyHuberLoss(threshold=0.7))
model.fit(x,y,epochs=100)

In [43]:
model.predict([10])

array([[0.34618336]], dtype=float32)

### Contrastive Loss

In [5]:
def contrastive_loss_with_margin(margin):
    def contrastive_loss(y_true, y_pred):
        square_pred = K.square(y_pred)
        margin_square = K.square(K.maximum(margin - y_pred, 0))
        return (y_true * square_pred + (1 - y_true) * margin_square)
    return contrastive_loss